In [2]:
import os
import numpy as np
import h5py
from scipy import stats
import scipy.io
import mne
import math 
mne.set_log_level('error')

from random import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.utils import shuffle

from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score


import torch
import torch.nn as nn
import torch.optim as optim
from torchsummary import summary

import optuna


from utils.load import Load
from config.default import cfg

%load_ext autoreload
%autoreload 2


In [3]:
device_name = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device_name)
print(device)

cuda


In [4]:
subject_data = {}
# Load the data  from the HDF5 file
target_dir = 'features'
tag = 'reproduced_with_bad'

for subject in cfg['subjects']:
    file_path = os.path.join(target_dir, tag+'_'+subject + '.h5')

    data = {}
    with h5py.File(file_path, 'r') as h5file:
        for key in h5file.keys():
            data[key] = np.array(h5file[key])

    subject_data[subject] = data


for subject_id in subject_data:
    print(subject_id)
    print(subject_data[subject_id].keys())

S1
dict_keys(['index', 'little', 'middle', 'ring', 'thumb'])
S2
dict_keys(['index', 'little', 'middle', 'ring', 'thumb'])
S3
dict_keys(['index', 'little', 'middle', 'ring', 'thumb'])
S4
dict_keys(['index', 'little', 'middle', 'ring', 'thumb'])
S5
dict_keys(['index', 'little', 'middle', 'ring', 'thumb'])


In [76]:
# 250 samples per subject
# 1250 total samples

In [15]:
class CustomDataset(Dataset):
    def __init__(self, subject_data, train_percent=0.8, seed=42, device=None, is_train=True):
        self.device = device
        self.is_train = is_train
        self.train_X, self.train_y, self.test_X, self.test_y = self.preprocess_data(subject_data, train_percent, seed)


    def preprocess_data(self, subject_data, train_percent, seed):
        np.random.seed(seed)
        global_train_features = []
        global_train_labels = []
        global_test_features = []
        global_test_labels = []

        for s, subject_id in enumerate(subject_data):
            data = subject_data[subject_id]

            for i, finger in enumerate(data):
                finger_data = data[finger]

                # Reshape
                finger_data = finger_data.reshape(finger_data.shape[0], -1)

                # Normalize (uncomment if needed)
                # finger_data = StandardScaler().fit_transform(finger_data)

                np.random.shuffle(finger_data)

                # To GPU
                finger_data = torch.tensor(finger_data).to(torch.float32).to(self.device)
                features = [(finger_data[d], ids[d]) for d in range(len(finger_data))]

                ids = torch.tensor(np.ones((len(finger_data))) * s).to(torch.int64).to(self.device)
                labels = torch.tensor(np.ones((len(finger_data))) * i).to(self.device)

                # Split
                train_features = features[:int(len(finger_data) * train_percent)]
                train_labels = labels[:int(len(finger_data) * train_percent)]
                test_features = features[int(len(finger_data) * train_percent):]
                test_labels = labels[int(len(finger_data) * train_percent):]

                global_train_features.extend(train_features)
                global_train_labels.extend(train_labels)
                global_test_features.extend(test_features)
                global_test_labels.extend(test_labels)

        return global_train_features, global_train_labels, global_test_features, global_test_labels

    
    def __len__(self):
        return len(self.train_y) if self.is_train else len(self.test_y)

    def __getitem__(self, idx):
        if self.is_train:
            return self.get_train_item(idx)
        else:
            return self.get_test_item(idx)

    def get_train_item(self, idx):
        features = self.train_X[idx][0]
        subject_id = self.train_X[idx][1]
        label = self.train_y[idx]

        return (features, subject_id), label

    def get_test_item(self, idx):
        features = self.test_X[idx][0]
        subject_id = self.test_X[idx][1]
        label = self.test_y[idx]

        return (features, subject_id), label




train_dataset = CustomDataset(subject_data, device=device, is_train=True)
test_dataset = CustomDataset(subject_data, device=device, is_train=False)

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=True)

In [89]:
# feature = ([EEG], [subject_id])
# label = finger

In [16]:
for i, (feature, label) in enumerate(train_dataloader):
    print(feature[0].shape)
    print(feature[1].shape)
    print(label.shape)
    print('---------------')
    break

torch.Size([16, 8216])
torch.Size([16])
torch.Size([16])
---------------


In [17]:
class CrossAttention(nn.Module):
    def __init__(self, feature_dim, user_dim, hidden_dim, feautre_hidden_dim,  num_classes):
        super(CrossAttention, self).__init__()
        self.feature_dim = feature_dim
        self.user_dim = user_dim
        self.hidden_dim = hidden_dim
        self.feature_hidden_dim = feautre_hidden_dim
        self.num_queries = 3

        self.layers = nn.ModuleDict({
            'feature extractor' : nn.Linear(self.feature_dim,  self.feature_hidden_dim),
            'batch norm': nn.BatchNorm1d(self.feature_hidden_dim),
            'query layer' : nn.Linear( self.feature_hidden_dim, self.feature_hidden_dim * self.hidden_dim ),  # Query transformation
            'key layer' : nn.Linear(self.user_dim, self.hidden_dim *self.user_dim ),  # Key transformation
            'value layer': nn.Linear(self.user_dim, self.hidden_dim *self.user_dim ),  # Value transformation
            'dropout': nn.Dropout(0.2),
            'classifier' : nn.Linear(self.feature_hidden_dim * self.hidden_dim, num_classes)
        })
        self.softmax = nn.Softmax(dim=2)

    def forward(self, features, user_indices):
        # Transform features to hidden_dim
        features = self.layers['feature extractor'](features)  
        features = self.layers['batch norm'](features)
       
        # Convert user_indices to one_hot vectors
        user_one_hot = torch.zeros(user_indices.size(0), self.user_dim, device=user_indices.device)
        user_one_hot.scatter_(1, user_indices.unsqueeze(1), 1)
     



        # Transform user_one_hot to query
        query = self.layers['query layer'](features)  
        query = query.view(-1, self.feature_hidden_dim, self.hidden_dim )  


        # Transform features to keys and values
        keys = self.layers['key layer'](user_one_hot)  
        keys = keys.view(-1,  self.hidden_dim, self.user_dim)  
        values = self.layers['value layer'](user_one_hot) 
        values = values.view(-1,  self.user_dim, self.hidden_dim) 
        # print(f'query: {query.shape}')
        # print(f'keys: {keys.shape}')
        # print(f'values: {values.shape}')

       
        # Calculate attention scores
        attention_scores = torch.bmm(query, keys) 

        # Normalize the attention scores
        attention_scores = attention_scores / math.sqrt(self.hidden_dim)

        attention_probs = self.softmax(attention_scores) 
     
        # Calculate the attended features
        attended_features = torch.bmm(attention_probs, values) 
        attended_features = attended_features.view(-1, self.feature_hidden_dim * self.hidden_dim)
        
        # Dropout
        attended_features = self.layers['dropout'](attended_features)
        
        # Classify the attended features
        output = self.layers['classifier'](attended_features)

        return output


In [32]:
# Hyperparameters
input_size = 8216
hidden_dim = 2
feautre_hidden_dim = 8

num_subjects = 5
output_size = 5
learning_rate = 1e-3

epochs = 200


In [36]:
# Create model and pass data
model = CrossAttention(input_size, num_subjects, hidden_dim, feautre_hidden_dim, output_size)
model.to(device)
summary(model, input_size=(5, 10, input_size));

Layer (type:depth-idx)                   Param #
├─ModuleDict: 1-1                        --
|    └─Linear: 2-1                       65,736
|    └─BatchNorm1d: 2-2                  16
|    └─Linear: 2-3                       144
|    └─Linear: 2-4                       60
|    └─Linear: 2-5                       60
|    └─Dropout: 2-6                      --
|    └─Linear: 2-7                       85
├─Softmax: 1-2                           --
Total params: 66,101
Trainable params: 66,101
Non-trainable params: 0


In [37]:
def accuracy(dataloader):
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():
        for features,  labels in dataloader:

            outputs = model(features = features[0], user_indices = features[1])
            _, predicted = torch.max(outputs.data, 1)
            total_predictions += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()

    accuracy = correct_predictions / total_predictions

    return accuracy * 100
    #print(f"Accuracy: {accuracy * 100:.2f}% ({correct_predictions}/{total_predictions})")

In [38]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    epoch_loss = 0.0

    for batch_features, batch_labels in train_dataloader:
        optimizer.zero_grad()
        outputs = model(features = batch_features[0], user_indices = batch_features[1])
        loss = criterion(outputs, batch_labels.long())
          
        # Backward propagation
        loss.backward()
        # Update the weights
        optimizer.step()

        epoch_loss += loss.item()

   

    if epoch % 10 == 9:
        train_accuracy = accuracy(train_dataloader)
        test_accuracy = accuracy(test_dataloader)
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss}, Train accuracy: {train_accuracy:.2f}%, Test accuracy: {test_accuracy:.2f}%")

print("#"*50)
print(f'Final_loss: {epoch_loss}')
print(f'Final train accuracy: {accuracy(train_dataloader):.2f}%')
print(f'Final test accuracy: {accuracy(test_dataloader):.2f}%')

Epoch 10/200, Loss: 56.08420732617378, Train accuracy: 68.80%, Test accuracy: 58.80%
Epoch 20/200, Loss: 20.382360510528088, Train accuracy: 89.10%, Test accuracy: 74.40%
Epoch 30/200, Loss: 11.554575692862272, Train accuracy: 96.50%, Test accuracy: 79.20%
Epoch 40/200, Loss: 8.316255658399314, Train accuracy: 95.60%, Test accuracy: 81.60%
Epoch 50/200, Loss: 6.187756382394582, Train accuracy: 98.10%, Test accuracy: 82.80%
Epoch 60/200, Loss: 4.921193789690733, Train accuracy: 91.90%, Test accuracy: 78.40%
Epoch 70/200, Loss: 3.416063972050324, Train accuracy: 98.50%, Test accuracy: 84.40%
Epoch 80/200, Loss: 1.2954519583145157, Train accuracy: 97.50%, Test accuracy: 83.60%
Epoch 90/200, Loss: 3.3669168053893372, Train accuracy: 99.00%, Test accuracy: 82.80%
Epoch 100/200, Loss: 1.2739150493289344, Train accuracy: 99.80%, Test accuracy: 84.80%
Epoch 110/200, Loss: 2.986518845485989, Train accuracy: 99.80%, Test accuracy: 84.00%
Epoch 120/200, Loss: 2.827765849826392, Train accuracy: 99